## File search assistant
The McKessonMDTranscripts demonstrates the creation and execution of an assistant using the file_search feature with a knowledge base based on earnings call transcripts downloaded from the internet.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [3]:
#r "nuget: AntRunnerLib, 0.6.13"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.6.13

## Ensure the McKessonMDTranscripts assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\McKessonMDTranscripts" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"McKessonMDTranscripts" contains the following files:
```
│   manifest.json
│   prompt.md
│
└───VectorStores
    └───McKessonMDTranscripts
            211101-MCK-Q2FY22-Earnings-Call-Transcript.md
            220202-MCK-Q3FY22-Earnings-Call-Transcript.md
            220505-MCK-Q4FY22-Earnings-Call-Transcript.md
            220803-MCK-Q1FY23-Earnings-Call-Transcript.md
            230508-MCK-Q4FY23-Earnings-Call-Transcript.md
            230802-MCK-Q1FY24-Earnings-Call-Transcript.md
            231101-MCK-Q2FY24-Earnings-Call-Transcript.md
            MCK-Q1FY21-Transcript.md
            MCK-Q1FY22-Transcript.md
            MCK-Q2-FY23-Earnings-Transcript.md
            MCK-Q2FY21-Transcript.md
            MCK-Q3-FY24-Earnings-Transcript.md
            MCK-Q4-FY24-Earnings-Call-Transcript.md
            MCK-Q4FY21-Transcript.md
            MCK-US-20230201-2761065-C.md
            Q1-FY19-Earnings-Call-Transcript.md
            Q1-FY20-Earnings-Call-Transcript.md
            Q2-FY19-Earnings-Call-Transcript.md
            Q2-FY20-Earnings-Call-Transcript.md
            Q3-FY19-Earnings-Call-Transcript.md
            Q3FY20-MCK-Earnings-Call-Transcript.md
            Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md
            Q4-FY19-Earnings-Call-Transcript.md
            Q4FY20-MCK-Corrected-Transcript.md
```

### Explanation
`AssistantUtility.Create` will create the vector store if necessary, upload the files, and then update the manifest with the vector store ID before creating the assistant.

In [4]:
var assistantId = await AssistantUtility.Create("McKessonMDTranscripts", config);
Console.WriteLine(assistantId)

asst_T3vIKP30dR8ioH1h3oDGPZdI


## Run the Assistant

`output.Dialog` shows the conversation.

In [5]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "McKessonMDTranscripts",
    Instructions = "How did covid impact results in Q3 2022?",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog)


User: How did covid impact results in Q3 2022?

Assistant: I searched my knowledge base for the answer.

In Q3 2022, the COVID-19 pandemic had a significant impact on McKesson's results. The emergence and spread of the Omicron variant led to unexpected fluctuations in volume and utilization trends, which initially showed signs of recovery but were disrupted by the new variant in December【4:0†source】【4:4†source】.

McKesson's US Pharmaceutical segment experienced a 12% growth in adjusted operating profit, largely driven by the distribution of COVID-19 vaccines and increased specialty volume. By January 31, the company had successfully distributed over 370 million Moderna and Johnson & Johnson COVID-19 vaccines across the United States【4:4†source】.

In the Medical-Surgical Solutions segment, revenues were positively impacted by the kitting, storage, and distribution of ancillary supplies for COVID-19 vaccines, although this was partially offset by lower revenue from COVID-19 tests compar

In [6]:
Console.WriteLine(output.LastMessage)

In Q3 2022, the COVID-19 pandemic had a significant impact on McKesson's results. The emergence and spread of the Omicron variant led to unexpected fluctuations in volume and utilization trends, which initially showed signs of recovery but were disrupted by the new variant in December【4:0†source】【4:4†source】.

McKesson's US Pharmaceutical segment experienced a 12% growth in adjusted operating profit, largely driven by the distribution of COVID-19 vaccines and increased specialty volume. By January 31, the company had successfully distributed over 370 million Moderna and Johnson & Johnson COVID-19 vaccines across the United States【4:4†source】.

In the Medical-Surgical Solutions segment, revenues were positively impacted by the kitting, storage, and distribution of ancillary supplies for COVID-19 vaccines, although this was partially offset by lower revenue from COVID-19 tests compared to the prior year【4:9†source】. 

Overall, McKesson's strong operational performance and contributions f

## Annotations

In [7]:
foreach(var annotation in output.Annotations)
{
    Console.WriteLine(annotation.FileCitation.FileId);
}

assistant-fjWqtpdCyNecNYDdGK9vhrG4
assistant-fjWqtpdCyNecNYDdGK9vhrG4
assistant-fjWqtpdCyNecNYDdGK9vhrG4
assistant-fjWqtpdCyNecNYDdGK9vhrG4
assistant-fjWqtpdCyNecNYDdGK9vhrG4


## Clean Up

In [8]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "McKessonMDTranscripts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
